### Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
import requests


### Definindo função que irá trabalhar com as colunas em JSON

In [2]:
def extract_key_value(row, column_name, key_name):
    data_list = json.loads(row[column_name])
    return [data[key_name] for data in data_list]

### Carregando os Datasets

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

print(f'movies{movies.shape}')
print(f'credits{credits.shape}')

movies(4803, 20)
credits(4803, 4)


In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


### Tratamento de dados

In [5]:
#Tratamento na tabela de Filmes

# Extraindo genero do Json e apagando coluna antiga
movies['GENERO'] = movies.apply(lambda row: extract_key_value(row, 'genres', 'name'), axis=1)
movies = movies.drop('genres',axis= 1 )

# Extraindo palavras chaves do Json e apagando coluna antiga
movies['KEYWORDS'] = movies.apply(lambda row: extract_key_value(row, 'keywords', 'name'), axis=1)
movies = movies.drop('keywords',axis= 1 )

# Extraindo produtora do Json e apagando coluna antiga
movies['PRODUTORA'] = movies.apply(lambda row: extract_key_value(row, 'production_companies', 'name'), axis=1)
movies = movies.drop('production_companies',axis= 1 )

# Extraindo país do Json e apagando coluna antiga
movies['PAIS'] = movies.apply(lambda row: extract_key_value(row, 'production_countries', 'name'), axis=1)
movies = movies.drop('production_countries',axis= 1 )

# Extraindo lingua do Json e apagando coluna antiga
movies['LINGUA'] = movies.apply(lambda row: extract_key_value(row, 'spoken_languages', 'name'), axis=1)
movies = movies.drop('spoken_languages',axis= 1 )

colunas_drop = ['homepage','release_date','revenue','runtime','vote_average','vote_count']
movies = movies.drop(colunas_drop, axis = 1)

In [6]:
movies.head(3)

,budget,id,original_language,original_title,overview,popularity,status,tagline,title,GENERO,KEYWORDS,PRODUTORA,PAIS,LINGUA
0,237000000,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Released,Enter the World of Pandora.,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]","[English, Español]"
1,300000000,285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],[English]
2,245000000,206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Released,A Plan No One Escapes,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]","[Français, English, Español, Italiano, Deutsch]"


In [7]:
#Tratamento na tabela de créditos

credits['PERSONAGENS'] = credits.apply(lambda row: extract_key_value(row, 'cast', 'character'), axis=1)
credits = credits.drop('cast',axis= 1 )

credits = credits.drop('crew', axis = 1)

### Juntando as duas tabelas

In [8]:
movies_credits = pd.merge(movies,credits, left_on= 'id', right_on= 'movie_id')
movies_credits.drop(['title_y','title_x','movie_id'], axis = 1, inplace= True)

Nessa etapa cria-se uma coluna única com todas as palavras presentes em todas as colunas da linha, com isso podemos calcular a similaridade dos filmes posteriormente

In [9]:
from typing import Any

def concat_columns(row):
    text = ' '.join(str(value) for value in row if isinstance(value, (str, list, dict)))
    return text.strip()

movies_credits['all_text'] = movies_credits.apply(concat_columns, axis=1)

Selecionando as colunas

In [10]:
movies_credits = movies_credits[['id','original_title','popularity','all_text']]

### Vetorização da coluna de palavras dos filmes

Primeiramente, é utilizado o objeto TfidfVectorizer para transformar o texto dos filmes em uma matriz numérica, que representa a frequência de cada palavra em cada texto. A partir disso, é calculada a similaridade entre os textos utilizando a medida de similaridade cosseno.

Em seguida, é utilizado o objeto CountVectorizer para criar uma nova matriz numérica, considerando apenas as frequências de cada palavra, ignorando a importância delas no contexto geral. A partir disso, também é calculada a similaridade cosseno entre os textos.

Por fim, é realizada a combinação das similaridades obtidas em cada etapa, utilizando a multiplicação das matrizes numéricas correspondentes. O resultado final é uma matriz de similaridade que leva em conta tanto a importância das palavras quanto a frequência delas nos textos dos filmes, permitindo uma análise mais precisa das similaridades entre os filmes.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

tfid = TfidfVectorizer()

vectorTfid = tfid.fit_transform(movies_credits['all_text']).toarray()
similarityTfidfVect = cosine_similarity(vectorTfid)

cv = CountVectorizer(stop_words='english', lowercase=True)

vector = cv.fit_transform(movies_credits['all_text']).toarray()
similarityCountVect = cosine_similarity(vector)

# COMBINING SIMILARITIE
res = np.multiply(similarityTfidfVect, similarityCountVect)

In [12]:
res 

array([[1.00000000e+00, 1.89169346e-04, 7.29948106e-03, ...,
        2.93116410e-04, 1.26039019e-04, 1.13305061e-04],
       [1.89169346e-04, 1.00000000e+00, 4.65127054e-04, ...,
        2.03999723e-03, 1.36276563e-03, 1.07934888e-03],
       [7.29948106e-03, 4.65127054e-04, 1.00000000e+00, ...,
        6.96098646e-04, 5.88605887e-04, 2.72228079e-04],
       ...,
       [2.93116410e-04, 2.03999723e-03, 6.96098646e-04, ...,
        1.00000000e+00, 4.52286537e-03, 4.47012415e-03],
       [1.26039019e-04, 1.36276563e-03, 5.88605887e-04, ...,
        4.52286537e-03, 1.00000000e+00, 4.59115945e-03],
       [1.13305061e-04, 1.07934888e-03, 2.72228079e-04, ...,
        4.47012415e-03, 4.59115945e-03, 1.00000000e+00]])

### Gerando as recomendações

No exemplo abaixo foi utilizado o filme Shrek para ser a referência, e a partir disso o modelo gerou 10 recomendações de outros filmes parecidos com a referência.

In [13]:
# Escolha um filme de referência
reference_movie = 'Shrek'

# Encontre o índice do filme de referência no dataframe
reference_index = movies_credits[movies_credits['original_title'] == reference_movie].index[0]

# Calcule as similaridades entre o filme de referência e todos os outros filmes no dataframe
similarities = res[reference_index]

# Ordene os filmes com base na similaridade calculada
similar_indices = similarities.argsort()[::-1]

# Retorne os N filmes mais similares ao filme de referência como recomendações
Quantidade_de_recomendacoes = 10
recommended_movies = movies_credits.loc[similar_indices[1:Quantidade_de_recomendacoes+1], 'original_title']
print(f'Filmes recomendados para quem gostou de {reference_movie}:')
print(recommended_movies)


Filmes recomendados para quem gostou de Shrek:
106                      Shrek the Third
565                              Shrek 2
86                   Shrek Forever After
42                           Toy Story 3
40                                Cars 2
347    Cloudy with a Chance of Meatballs
88                            Big Hero 6
187                        Puss in Boots
152                      Kung Fu Panda 3
566                                 Cars
Name: original_title, dtype: object
